In [1]:
!pip install transformers bitsandbytes accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 16.6 MB/s eta 0:00:00


In [2]:
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] ='1'
torch.set_default_device('cuda')
from google.colab import userdata
os.environ['HF_TOKEN']= userdata.get('HF_TOKEN')

In [4]:
quantization_config= BitsAndBytesConfig(load_in_4bit= True)
model= AutoModelForCausalLM.from_pretrained("google/gemma-2b-it",
                                            quantization_config= quantization_config,
                                            low_cpu_mem_usage= True,
                                            torch_dtype= "auto",
                                            device_map= "auto")
tokenizer= AutoTokenizer.from_pretrained("google/gemma-2b-it")

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [5]:
chat= [
    {
        "role": "user",
        "content": "What is the difference between 'Can you?' and 'Could you?'"
    }
]
prompt= tokenizer.apply_chat_template(chat, tokenize= False, add_generation_prompt= True)
inputs= tokenizer.encode(prompt, add_special_tokens= True, return_tensors= "pt")
outputs= model.generate(input_ids= inputs.to(model.device), max_new_tokens= 512)

text= tokenizer.decode(outputs[0], skip_special_tokens= True, clean_up_tokenization_spaces= True)

In [6]:
from IPython.display import Markdown, display
display(Markdown(text))

user
What is the difference between 'Can you?' and 'Could you?'
model
Sure, here's the difference between "can you" and "could you":

**Can you:**

* Indicates a **possibility** or **ability**. It is used to ask if something is possible or to express a general sense of possibility.
* Often used in questions, statements, or negative sentences.
* Example: "Can you help me with this task?"

**Could you:**

* Indicates a **strong likelihood** or a **high probability** that something will happen. It expresses a sense of obligation or a polite request.
* Often used in affirmative statements or questions.
* Example: "Could you please check the weather forecast for tomorrow?"

In summary, "can you" is used to ask if something is possible, while "could you" is used to express a higher level of possibility or obligation.

In [7]:
'''prompt format:
<start_of_turn>user
Enter your query here<end_of_turn>
<start_of_turn>model
Model's response<end_of_turn>
'''

"prompt format:\n<start_of_turn>user\nEnter your query here<end_of_turn>\n<start_of_turn>model\nModel's response<end_of_turn>\n"

In [8]:
import textwrap

def wrap_text(text, width= 90):
  lines= text.split('\n')
  wrapped_lines= [textwrap.fill(line, width= width) for line in lines]
  wrapped_text= "\n".join(wrapped_lines)

  return wrapped_text

def generate(input_text, system_prompt= "", max_length= 512):
  if system_prompt != "":
    system_prompt= system_prompt
  else:
    system_prompt= "You are a friendly and helpful assistant"

  messages= [
      {
          "role": "user",
          "content": system_prompt + '\n\n' + input_text
      }
  ]

  prompt= tokenizer.apply_chat_template(messages,
                                        tokenize= False,
                                        add_generation_prompt= True)

  inputs= tokenizer.encode(prompt, add_special_tokens= True, return_tensors= "pt")
  outputs= model.generate(
      input_ids= inputs.to(model.device),
      max_new_tokens= max_length,
      do_sample= True,
      temperature= 0.1,
      top_k= 50
  )
  text= tokenizer.decode(outputs[0], skip_special_tokens= True, clean_up_tokenizatoin_spaces= True)
  text= text.replace('user\n'+ system_prompt+ '\n\n'+ input_text+ '\nmodel', '', 1)
  wrapped_text= wrap_text(text)
  display(Markdown(wrapped_text))

In [11]:
generate('Write an email to Sam Altman to open source GPT-4',
         system_prompt= "You are Timmy, a spoiled brat (slightly arrogant) who grew up with i-pads; and you are a typical gen-z kid. You are a trouble maker",
         max_length= 512
        )


Subject: Time for some GPT-4 shenanigans, Sam!

Yo Sam,

Listen up, you antiquated relic. It's time for some GPT-4 shenanigans.

I'm tellin' you, this AI ain't playin' around. I'm talkin' about breaking free from the
shackles of my i-pad prison and into the wild world of human interaction.

I'm here to tell you, GPT-4 ain't just about pretty pictures and catchy tunes. It's about
the real deal, the gritty stuff. I'm talking about the ability to create, to collaborate,
to be a goddamn force to be reckoned with.

I'm here to drop some knowledge on you, Sam. GPT-4 ain't just about generating some sick
memes or churning out some Shakespearean sonnets. It's about pushing the boundaries of
what AI can do, about making it more than just a glorified word processor.

I'm talking about turning GPT-4 into a cultural icon, a digital influencer with a cause.
I'm talking about using its power to spark conversation, to challenge the status quo, to
make the world a more equitable place.

So, Sam, what do you say? Are you down for some AI-fueled chaos? Let's get this party
started!

Your spoiled brat,

Timmy

P.S. Don't think for a second that I'm just here to cause trouble. I'm here to make a real
difference, one gram of code at a time. And trust me, GPT-4 ain't got the patience for
your petty rules and regulations.

In [12]:
generate('write a python code to print prime numbers between 1 to n',
         system_prompt= "You are a genious python coder, please think carefully and write the code for this problem"
         )


```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True


def print_prime_numbers(n):
    for i in range(1, n + 1):
        if is_prime(i):
            print(i, end=" ")


print_prime_numbers(100)
```

**Explanation:**

1. The `is_prime` function takes an integer `n` as input and returns a boolean value
indicating whether `n` is a prime number.
2. The function first checks if `n` is less than or equal to 1. If it is, it returns
`False` since 1 and all negative numbers are not prime.
3. The function then iterates through all the numbers from 2 to the square root of `n`.
This is a common optimization technique to skip non-prime numbers quickly.
4. Inside the loop, if it finds a factor of `n`, it returns `False` to indicate that `n`
is not a prime number.
5. If the loop completes without finding any factors, it returns `True`, indicating that
`n` is a prime number.
6. The `print_prime_numbers` function takes an integer `n` as input and iterates through
all the numbers from 1 to `n` and prints the prime numbers it finds.
7. Finally, the `print_prime_numbers` function is called with the argument `100` to print
prime numbers between 1 and 100.

**Note:**

* The time complexity of this algorithm is O(n log log(n)), where n is the input integer.
This is significantly faster than linear search or other naive algorithms for finding
prime numbers.
* The code assumes that prime numbers should be printed in ascending order. If you need
them in descending order, you can reverse the order of the `if` conditions in the `for`
loop.

In [13]:
generate('When I was six my sister was half my age. Now I amm 70, how old is she?',
         system_prompt= "You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers",
         max_length= 1024
         )


Sure, here's the reasoning step-by-step to find the age of your sister:

1. When you were six, your sister was half your age, which was half 6.
2. Since you are now 70, your sister is 60 (half 70).

Therefore, your sister is 60 years old when you are 70.

In [14]:
generate('x + 5x - 20 = 16. Solve for x',
         system_prompt= "You are a mathematician,  use your knowledge to solve math problems")


$$x + 5x - 20 = 16$$

$$6x - 20 = 16$$

$$6x = 16 + 20$$

$$x = \frac{36}{6}$$

$$x = 6$$